In [68]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np
    #database

df_qv = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\QV.xlsx')
df_stock = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\stock.xlsx') 
df_stock.drop([0])
df_inventory = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\inventory.xlsx')
df_WC = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\fixed_doc\Warehouse_Risk_Category.xlsx')
df_CS = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\CS.xlsx')
df_CP = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\fixed_doc\CP_approved.xlsx')
df_movement = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\movement.xlsx')
df_vessel = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\inventory2.xlsx')
df_invoice = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\invoice.xlsx')
df_ffca= pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\FFCA.xlsx' )


In [69]:
# df_invoice = pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0301\invoice.xlsx')
df_input= pd.read_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0308\input.xlsx')


def sales(a):
    return a == "sales"
df4 = df_input.loc[df_input['status'].apply(sales)]

In [89]:


df_pref = pd.merge(df4, df_ffca, left_on='Purchase Assignment Reference', right_on='Purchase Assignment ID', how='left')
cols=['Purchase Assignment Reference', 'Sales Assignment ID ']
df4_x = df4
df_sref = pd.DataFrame(df_pref, columns=cols)

dfnan = df_sref[df_sref['Sales Assignment ID '].isnull()]
dfnonnan = df_sref[df_sref['Sales Assignment ID '].notnull()]

df_candidates = pd.merge(dfnonnan, df_ffca, on='Sales Assignment ID ', how='left')
df_candidates = pd.concat([df_candidates, dfnan])
# # df_candidates = df_candidates[df_candidates['Financing Bank facility']== 'CONCS BB ING']
df_candidates['status'] = 'sales'
cols_candidate = ['Purchase Assignment Reference', 'status']

df_candidates = pd.DataFrame(df_candidates, columns=cols_candidate)
df_candidates = df_candidates.drop_duplicates(['Purchase Assignment Reference'])
# df_candidates.columns = ['Purchase Assignment Reference', 'status']
df4 = df_candidates



#column filter
cols_ffca = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
   'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
      'Sales Invoiced Amount', 'Sales Settled Amount', 'QTY']
cols_qv = ['Assignment ID', 'Group Company', 'Trade ID', 'Quota ID', 'Traffic Operator']
cols_inventory = ['INVENTORY ID', 'PURCHASE ASSIGNMENT', 'DRY WEIGHT', 'WET WEIGHT', 'EXPECTED RECEIPT DATE']
cols_stock = ['Inventory Id','Origin Country']
cols_cs = ['ID', 'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank', 'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount']
cols_ffca_blend = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity', 'Inventory ID','Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
   'Purchase Settled Amount', 'Authorised Paid Quantity', 'Financed Quantity', 'Current Value', 'Treasury Payment Date', 'Purchase Invoice', 'Purchase Invoiced Amount', \
      'Sales Invoiced Amount', 'Sales Settled Amount', 'QTY']
cols_invoice = ['Assignment Ref', 'Invoice Type', 'Invoice Date', 'Payment Expected Due Date', 'Days Overdue', 'Invoice No', 'Unpaid Amount ', 'Invoice Amount']
cols_ar = ['Sales Assignment ID ', 'Accounts Receivable']

df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)
df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory)
df_stock_f1 = pd.DataFrame(df_stock, columns=cols_stock)
df_CS_f1 = pd.DataFrame(df_CS, columns=cols_cs)
df_invoice_f1 = pd.DataFrame(df_invoice, columns=cols_invoice)
df_ffca_f1['Current Value'] = pd.to_numeric(df_ffca_f1['Current Value'], errors='coerce')
df_ffca_f1['Authorised Paid Quantity'] = pd.to_numeric(df_ffca_f1['Authorised Paid Quantity'], errors='coerce')
df_ffca_f1['Financed Quantity'] = pd.to_numeric(df_ffca_f1['Financed Quantity'], errors='coerce'
                                                )
df_invoice_f1['Unpaid Amount '] = df_invoice_f1['Unpaid Amount '].str.replace(',', '').astype(str)
df_invoice_f1['Unpaid Amount '] = pd.to_numeric(df_invoice_f1['Unpaid Amount '], errors='coerce')
df_invoice_f1['Invoice Amount'] = df_invoice_f1['Invoice Amount'].str.replace(',', '').astype(str)
df_invoice_f1['Invoice Amount'] = pd.to_numeric(df_invoice_f1['Invoice Amount'], errors='coerce')

df_invoice_f1 = df_invoice_f1.loc[(df_invoice_f1['Invoice Type'] == 'FINAL') | (df_invoice_f1['Invoice Type'] == 'PROVISIONAL')]

df_invoice_f1['Unpaid Amount '].replace('-', 0, inplace = True)
df_invoice_f1['Paid Amount'] = df_invoice_f1['Invoice Amount'] - df_invoice_f1['Unpaid Amount ']
df_ar = pd.DataFrame(df_ffca, columns=cols_ar)

In [90]:
#sales
#delete nan sales
df_ffca_f1.dropna(subset=['Sales Assignment ID '], inplace=True)
# 一、先将字段拆分
df_CS_f1['Assignments']=df_CS['Assignments'].map(lambda x:x.split(','))
#二、然后直接调用explode()方法
df_CS_f1_explode = df_CS_f1.explode('Assignments')

def blend(a):
   return a == "Blend"

#blend
bdf_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca_blend)
df_blend_inventory = df_inventory_f1.loc[df_inventory_f1['PURCHASE ASSIGNMENT'].apply(blend)]

row_list = df_inventory_f1[df_inventory_f1['PURCHASE ASSIGNMENT'] == 'Blend'].index.tolist() # 获得含有该值的行的行号
df_inventory_f1 = df_inventory_f1.drop(row_list)

bffca_qv = pd.merge(bdf_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'left')
bffca_qv_cp = pd.merge(bffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
binventory_stock = pd.merge(df_blend_inventory, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'], how='left')
FQC = pd.merge(bffca_qv_cp, df_CS_f1_explode, left_on=["Sales Assignment ID "], right_on=["Assignments"], how = 'outer')
bfinal_sales = pd.merge(FQC, binventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])

def function(a, b, c):
   return c * (1 + (a-b)/a)

bfinal_sales['Purchased Wet Weight']  = bfinal_sales.apply(lambda x: function(x['WET WEIGHT'], x['DRY WEIGHT'], x['QTY']), axis = 1)

bfinal_sales = bfinal_sales.drop(columns = ['WET WEIGHT', 'DRY WEIGHT'])

#non-blend
ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on=["Purchase Assignment ID"], right_on=["Assignment ID"], how= 'left')
ffca_qv_cp = pd.merge(ffca_qv, df_CP, left_on=['Counterparty of Sales'], right_on=['Sales Assignment Counterparty'], how='outer')
inventory_stock = pd.merge(df_inventory_f1, df_stock_f1, left_on=['INVENTORY ID'], right_on=['Inventory Id'], how='left')
nFQC = pd.merge(ffca_qv_cp, df_CS_f1_explode, left_on=["Sales Assignment ID "], right_on=["Assignments"], how = 'outer')
nfinal_sales = pd.merge(nFQC, inventory_stock, left_on=['Inventory ID'], right_on=['INVENTORY ID'])
nfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
   'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
   'Sales Invoice ID', 'Estimated Sales Valuation',
   'Purchase Settled Amount', 'Authorised Paid Quantity',
   'Financed Quantity', 'Current Value', 'Treasury Payment Date',
   'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
   'Sales Settled Amount', 'QTY', 'Assignment ID', 'Group Company',
   'Trade ID', 'Quota ID', 'Traffic Operator',
   'Sales Assignment Counterparty',
   'Sales Contractual Counterparty Open Account Approved ', 'ID',
   'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
   'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
   'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
   'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

order = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity','Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales', 'Sales Invoice ID', 'Estimated Sales Valuation', \
   'Purchase Settled Amount', 'Authorised Paid Quantity','Financed Quantity', 'Current Value', 'Treasury Payment Date','Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount','Sales Settled Amount',  \
      'Assignment ID', 'Group Company','Trade ID', 'Quota ID', 'Traffic Operator','Sales Assignment Counterparty','Sales Contractual Counterparty Open Account Approved ', 'ID', \
   'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank','Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'QTY', 'Purchased Wet Weight',\
   'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

bfinal_sales = bfinal_sales[order]

bfinal_sales.columns = ['Purchase Assignment ID', 'Sales Assignment ID ', 'Commodity',
   'Inventory ID', 'Counterparty of Purchase', 'Counterparty of Sales',
   'Sales Invoice ID', 'Estimated Sales Valuation',
   'Purchase Settled Amount', 'Authorised Paid Quantity',
   'Financed Quantity', 'Current Value', 'Treasury Payment Date',
   'Purchase Invoice', 'Purchase Invoiced Amount', 'Sales Invoiced Amount',
   'Sales Settled Amount', 'Assignment ID', 'Group Company', 'Trade ID',
   'Quota ID', 'Traffic Operator', 'Sales Assignment Counterparty',
   'Sales Contractual Counterparty Open Account Approved ', 'ID',
   'Assignments', 'Term', 'Issuing/Collecting Bank', '1st Confirming Bank',
   'Opening Date', 'Expiry Date', 'Balance Amount', 'Maximum Amount',
   'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Purchase Dry Weight', 'Purchase Wet Weight',
   'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country']

final_sales = pd.concat([nfinal_sales,bfinal_sales])

final_sales = pd.merge(final_sales, df_invoice_f1, left_on=["Sales Assignment ID "], right_on=["Assignment Ref"], how='left')
final_sales.fillna(0)


# final_sales['Unpaid Amount ] = pd.to_numeric(final_sales['Unpaid Amount ], errors='coerce')
final_sales['Estimated Sales Valuation'] = pd.to_numeric(final_sales['Estimated Sales Valuation'], errors='coerce')
final_sales['Current Value'] = pd.to_numeric(final_sales['Current Value'], errors='coerce')

#Comparison between invoice amount and current value


def function1(a,b):
   if a == 'PROVISIONAL':
      return b
   else:
      return 0


def function2(a,b):
   if a == 'FINAL':
      return b
   else:
      return 0

def functionxx_3(a,b):
   if a > b:
      return a-b
   else:
      return 0


final_sales['Unpaid Provisional Receivables(USD)'] = final_sales.apply(lambda x: function1(x['Invoice Type'], x['Unpaid Amount ']), axis = 1)
final_sales['Unpaid Final Receivables(USD)'] = final_sales.apply(lambda x: function2(x['Invoice Type'], x['Unpaid Amount ']), axis = 1) #wrong
# final_sales.loc[:,"Uninvoiced Final Receivables(USD)"] = final_sales["Estimated Sales Valuation"] - final_sales["Invoice Amount"]
final_sales['Uninvoiced Final Receivables(USD)'] = final_sales.apply(lambda x: functionxx_3(x['Estimated Sales Valuation'], x['Invoice Amount']), axis = 1)


def function3(a,b):
   if a >= b:
      return b
   if a < b:
      return -a

def function4(a,b):
   if -a < b:
      return b + a
   if -a >= b:
      return 0

def function5(a,b):
   if a >= b:
      return b
   else:
      return a

def function6(a):
   if a == "FINAL":
      return "100%"
   else:
      return "90%"

final_sales['Total Financeable Sales Invoices Covered by LC']  = final_sales.apply(lambda x: function3(x['Invoice Amount'], x['Balance Amount']), axis = 1)
final_sales['LC Cover Available to SCR (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function4(x['Invoice Amount'], x['Balance Amount']), axis = 1)
final_sales['Sales Provisional Percentage']  = final_sales.apply(lambda x: function6(x['Invoice Type']), axis = 1)
final_sales['SCR Covered By LC (SCR-Sales Contractual Rights)']  = final_sales.apply(lambda x: function5(x['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], x['Uninvoiced Final Receivables(USD)']), axis = 1)


import numpy as np

#adding 
final_sales['Sales Provisional Number'] = '1'
final_sales['Authorised Paid Quantity'] = pd.to_numeric(final_sales['Authorised Paid Quantity'], errors='coerce')
final_sales['Financed Quantity'] = pd.to_numeric(final_sales['Financed Quantity'], errors='coerce')
final_sales.loc[:,"Financing Ratio"] = final_sales["Authorised Paid Quantity"]/final_sales["Financed Quantity"]

final_sales['Cost Code ID'] = 'I01'
final_sales['Invoice Line Quantity'] = '0'
final_sales['Invoice Line Quantity UOM'] = 'MT'
final_sales['Purchase Dry Weight UOM'] = 'MT'
final_sales['Purchase Wet Weight UOM'] = 'MT'
final_sales['Receipt In Progress'] = 'N'
final_sales['Delivery In Progress'] = 'N'
final_sales['Summary Outstanding Receivables(USD)'] = final_sales['Unpaid Provisional Receivables(USD)'] + final_sales['Unpaid Final Receivables(USD)'] + final_sales['Uninvoiced Final Receivables(USD)']
final_sales['Sales Invoice Counterparty'] = final_sales['Counterparty of Sales']
final_sales['Eligible Dry Weight'] = final_sales['Purchase Dry Weight']
final_sales['Eligible Wet Weight'] = final_sales['Purchase Wet Weight']


def function7(a, b):
   if b == 0:
      return 0
   else:
      return -a/b
      
def function8(a):
   if a>1:
      return 1
   else:
      return a
      


final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function7(x['Estimated Sales Valuation'], x['Invoice Amount']), axis = 1)

final_sales['Sales Assignment Eligible Percentage']  = final_sales.apply(lambda x: function8(x['Sales Assignment Eligible Percentage']), axis = 1)

final_sales = final_sales.drop(columns=['Inventory ID', 'Current Value','Purchase Settled Amount', 'Purchase Invoice', 'Purchase Invoiced Amount', 'Current Value', 'Treasury Payment Date', \
'Assignment ID', 'Sales Assignment Counterparty', 'Assignments', 'INVENTORY ID', 'PURCHASE ASSIGNMENT', 'Unpaid Amount ', \
'EXPECTED RECEIPT DATE', 'Inventory Id', 'Origin Country', 'QTY', 'Assignment Ref', 'Sales Invoice ID', 'Sales Invoiced Amount', 'Sales Settled Amount'], axis=1)


final_sales.columns = ['Purchase Assignment Reference', 'Sales Assignment Reference', 'Purchase Commodity',
   'Purchase Contractual Counterparty', 'Sales Assignment Counterparty', 
   'Cost Amount(USD)', 'Authorised Paid Quantity','Financed Quantity', 
   'Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Traffic Operator',
   'Sales Contractual Counterparty Open Account Approved', 'LC Reference', 'Sales Credit Security Type',
   'LC Opening Bank', 'LC Confirming Bank', 'LC Issuance Date', 'LC Expiry Date', 'LC Current Balance', 'LC Original Amount',
   'Purchase Dry Weight', 'Purchase Wet Weight', 
   'Sales Provisional Final', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue','Sales Invoice ID','Invoice Line Amount(USD)','Paid Amount', 'Unpaid Provisional Receivables(USD)',
   'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)',
   'Total Financeable Sales Invoices Covered by LC',
   'LC Cover Available to SCR (SCR-Sales Contractual Rights)',
   'Sales Provisional Percentage',
   'SCR Covered By LC (SCR-Sales Contractual Rights)',
   'Sales Provisional Number', 'Financing Ratio', 'Cost Code ID',
   'Invoice Line Quantity', 'Invoice Line Quantity UOM',
   'Purchase Dry Weight UOM', 'Purchase Wet Weight UOM',
   'Receipt In Progress', 'Delivery In Progress',
   'Summary Outstanding Receivables(USD)', 'Sales Invoice Counterparty',
   'Eligible Dry Weight', 'Eligible Wet Weight',
   'Sales Assignment Eligible Percentage']


order4 = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Sales Assignment Reference','Purchase Traffic Operator', \
   'Purchase Contractual Counterparty', 'Purchase Commodity', 'Sales Assignment Counterparty', 'Sales Credit Security Type', 'LC Reference', 'LC Original Amount', \
      'LC Current Balance', 'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank', 'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC', \
            'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID', 'Sales Provisional Final', \
               'Sales Provisional Number', 'Sales Provisional Percentage', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Sales Invoice Counterparty', \
                  'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount', 'Sales Assignment Eligible Percentage', \
                        'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)', \
                           'Summary Outstanding Receivables(USD)', 'Sales Contractual Counterparty Open Account Approved', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', \
                              'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

final_sales_test = final_sales[order4]


result4 = pd.merge(final_sales_test, df4, on=["Purchase Assignment Reference"], how='right')
result4['Cost Amount(USD)'] = pd.to_numeric(result4['Cost Amount(USD)'], errors='coerce')
result4['Paid Amount'] = pd.to_numeric(result4['Paid Amount'], errors='coerce')
result4 = result4.drop_duplicates(['Purchase Assignment Reference', 'Sales Invoice ID'])

a = result4.groupby('Sales Invoice ID')['Purchase Dry Weight'].transform('sum')
result4['%'] = (result4['Purchase Dry Weight']/result4.groupby('Sales Assignment Reference')['Purchase Dry Weight'].transform('sum'))

result4['Sales Assignment Eligible Percentage'] = result4['%']

result4['Unpaid Provisional Receivables(USD)'] = result4['Unpaid Provisional Receivables(USD)'] * result4['%']
result4['Unpaid Final Receivables(USD)'] = result4['Unpaid Final Receivables(USD)'] * result4['%']
result4['Uninvoiced Final Receivables(USD)'] = result4['Uninvoiced Final Receivables(USD)'] * result4['%']

result4 = pd.merge(result4, df_ar, left_on='Sales Assignment Reference', right_on='Sales Assignment ID ')
result4['Accounts Receivable'] = result4['Accounts Receivable'] * result4['%']
result4['Summary Outstanding Receivables(USD)'] = result4['Accounts Receivable']
result4 = result4.drop_duplicates(['Purchase Assignment Reference', 'Sales Invoice ID'])
result4['Uninvoiced Final Receivables(USD)'] = result4['Summary Outstanding Receivables(USD)'] - result4['Unpaid Final Receivables(USD)']  - result4['Unpaid Provisional Receivables(USD)']

result4 = result4.drop(columns = ['Sales Assignment ID ', 'Accounts Receivable'])

dfa = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Invoice Line Amount(USD)'].sum()
dfb = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Cost Amount(USD)'].sum()
dfc = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Paid Amount'].sum()
dfd = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Unpaid Provisional Receivables(USD)'].sum()
dfe = result4.groupby(['Purchase Assignment Reference', 'Sales Assignment Reference'])['Unpaid Final Receivables(USD)'].sum()
 
dfa.reset_index(drop=False)
dfb.reset_index(drop=False)
dfc.reset_index(drop=False)
dfd.reset_index(drop=False)
dfe.reset_index(drop=False)

dfa = pd.DataFrame(dfa)
dfb = pd.DataFrame(dfb)
dfc = pd.DataFrame(dfc)
dfd = pd.DataFrame(dfd)
dfe = pd.DataFrame(dfe)

df_1 = pd.merge(dfa,dfb, on = 'Purchase Assignment Reference')
df_2 = pd.merge(dfd,dfc, on = 'Purchase Assignment Reference')
df_3 = pd.merge(df_2,dfe, on = 'Purchase Assignment Reference')
df_4 = pd.merge(df_2,df_3, on = 'Purchase Assignment Reference')
df_5 = pd.merge(df_4,df_1, on = 'Purchase Assignment Reference')
df_5 = df_5.drop_duplicates(['Unpaid Provisional Receivables(USD)_x'])

df5 = df_5.reset_index()

order = ['Purchase Group Company Code', 'Purchase Trade ID', 'Purchase Quota Titan ID', 'Purchase Assignment Reference', 'Sales Assignment Reference','Purchase Traffic Operator', \
'Purchase Contractual Counterparty', 'Purchase Commodity', 'Sales Assignment Counterparty', 'Sales Credit Security Type', 'LC Reference', 'LC Original Amount', \
   'LC Current Balance', 'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank', 'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC', \
      'LC Cover Available to SCR (SCR-Sales Contractual Rights)', 'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID', 'Sales Provisional Final', \
            'Sales Provisional Number', 'Sales Provisional Percentage', 'Sales Invoice Date', 'Sales Invoice Due Date', 'Sales Invoice Days Overdue', 'Sales Invoice Counterparty', \
               'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity', 'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount', 'Sales Assignment Eligible Percentage', \
                  'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio', 'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)', 'Uninvoiced Final Receivables(USD)', \
                        'Summary Outstanding Receivables(USD)', 'Sales Contractual Counterparty Open Account Approved', 'Purchase Dry Weight', 'Purchase Dry Weight UOM', \
                           'Purchase Wet Weight', 'Purchase Wet Weight UOM', 'Receipt In Progress', 'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']
                           
dff = pd.merge(df5, result4, on='Purchase Assignment Reference', how='right')
dff1 = dff.drop(columns = ['Cost Amount(USD)_y', 'Paid Amount_y', 'Invoice Line Amount(USD)_y', 'Unpaid Final Receivables(USD)_y', 'Unpaid Provisional Receivables(USD)_y', 'Unpaid Provisional Receivables(USD)'])
dff2 = dff1.drop_duplicates(['Purchase Assignment Reference'])
dff3 = dff2.rename(columns={'Cost Amount(USD)_x':'Cost Amount(USD)', 'Paid Amount_x':'Paid Amount', 'Invoice Line Amount(USD)_x':'Invoice Line Amount(USD)', 'Unpaid Final Receivables(USD)_x':'Unpaid Final Receivables(USD)', 'Unpaid Provisional Receivables(USD)_x':'Unpaid Provisional Receivables(USD)'})
dff = dff3 [order]





dff['Sales Invoice ID'] = np.nan
dff['Sales Provisional Final '] = np.nan
dff['Sales Provisional Number'] = np.nan
dff['Sales Provisional Percentage'] = np.nan
dff['Sales Invoice Date'] = np.nan
dff['Sales Invoice Due Date'] = np.nan
dff['Sales Invoice Days Overdue'] = np.nan
dff['Sales Invoice Counterparty'] = np.nan
dff['Cost Code ID'] = np.nan
dff['Invoice Line Quantity'] = np.nan
dff['Invoice Line Quantity UOM'] = np.nan

def function_x(a,b,c):
   if a == c:
      return 0
   if a > c and b > (a-c):
      return (a-c) 
   if a > c and b < a:
      return b

def function_y(a,b,c):
   if a == c:
      return b
   if a > c and b > (a-c):
      return b - (a-c)
   if a > c and b < (a-c):
      return 0

def function_z(a,b):
   if a >= b:
      return b
   else:
      return a

dff = dff.loc[:, ~dff.columns.duplicated()]


def function_x1(a,b,c):
   if a == 0 or b == 0:
      return c == 0
   else:
      return c

def function_x2(a):
   if a == None:
      return a == 'Unknown'
   else:
      return a

dff['Total Financeable Sales Invoices Covered by LC']  = dff.apply(lambda x: function_x(x['Invoice Line Amount(USD)'], x['LC Current Balance'], x['Paid Amount']), axis = 1)
dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)']  = dff.apply(lambda x: function_y(x['Invoice Line Amount(USD)'], x['LC Current Balance'], x['Paid Amount']), axis = 1)
dff['Sales Provisional Percentage']  = dff.apply(lambda x: function6(x['Sales Provisional Final ']), axis = 1)
dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = pd.to_numeric(dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], errors='coerce') #chenge to float
dff['SCR Covered By LC (SCR-Sales Contractual Rights)']  = dff.apply(lambda x: function5(x['LC Cover Available to SCR (SCR-Sales Contractual Rights)'], x['Uninvoiced Final Receivables(USD)']), axis = 1)

dff["Financing Ratio"]  = dff.apply(lambda x: function_x1(x['Authorised Paid Quantity'], x['Financed Quantity'], x['Financing Ratio']), axis = 1)
dff['LC Confirming Bank']  = dff.apply(lambda x: function_x2(x['LC Confirming Bank']), axis = 1)

dff['Total Financeable Sales Invoices Covered by LC'].fillna(0, inplace=True)
dff['LC Cover Available to SCR (SCR-Sales Contractual Rights)'].fillna(0, inplace=True)
dff['SCR Covered By LC (SCR-Sales Contractual Rights)'].fillna(0, inplace=True)




dff = dff.rename(columns={'Sales Provisional Final ':'Sales Provisional Final', })



# dff['Summary Outstanding Receivables(USD)'] = dff['Unpaid Provisional Receivables(USD)'] + dff['Unpaid Final Receivables(USD)'] + dff['Uninvoiced Final Receivables(USD)']

dff = dff[order4]
dff.reset_index(drop=True)

result4['LC Current Balance'] = np.nan
result4['Sales Credit Security Type'] = np.nan
result4['LC Reference'] = np.nan
result4['LC Original Amount'] = np.nan
result4['LC Issuance Date'] = np.nan
result4['LC Expiry Date'] = np.nan
result4['LC Confirming Bank'] = np.nan
result4['Total Financeable Sales Invoices Covered by LC'] = np.nan
result4['LC Cover Available to SCR (SCR-Sales Contractual Rights)'] = np.nan
result4['SCR Covered By LC (SCR-Sales Contractual Rights)'] = np.nan
result4['Cost Amount(USD)'] = np.nan
result4['Authorised Paid Quantity'] = np.nan
result4['Financed Quantity'] = np.nan
result4['Uninvoiced Final Receivables(USD)'] = np.nan
result4['Summary Outstanding Receivables(USD)'] = np.nan
result4['Sales Contractual Counterparty Open Account Approved '] = np.nan
result4['Purchase Dry Weight'] = np.nan
result4['Purchase Dry Weight UOM'] = np.nan
result4['Purchase Wet Weight'] = np.nan
result4['Purchase Wet Weight UOM'] = np.nan
result4['Receipt In Progress'] = np.nan
result4['Delivery In Progress'] = np.nan
result4['Eligible Dry Weight'] = np.nan
result4['Eligible Wet Weight'] = np.nan

result4 = result4.drop(columns = ['%', 'status'])
result4 = result4[order4]
result4.reset_index(drop=True)

dff.columns = ['Purchase Group Company Code', 'Purchase Trade ID',
       'Purchase Quota Titan ID', 'Purchase Assignment Reference',
       'Sales Assignment Reference', 'Purchase Traffic Operator',
       'Purchase Contractual Counterparty', 'Purchase Commodity',
       'Sales Assignment Counterparty', 'Sales Credit Security Type',
       'LC Reference', 'LC Original Amount', 'LC Current Balance',
       'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank',
       'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC',
       'LC Cover Available to SCR (SCR-Sales Contractual Rights)',
       'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID',
       'Sales Provisional Final1', 'Sales Provisional Final2',
       'Sales Provisional Number', 'Sales Provisional Percentage',
       'Sales Invoice Date', 'Sales Invoice Due Date',
       'Sales Invoice Days Overdue', 'Sales Invoice Counterparty',
       'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity',
       'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount',
       'Sales Assignment Eligible Percentage',
       'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio',
       'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)',
       'Uninvoiced Final Receivables(USD)',
       'Summary Outstanding Receivables(USD)',
       'Sales Contractual Counterparty Open Account Approved',
       'Purchase Dry Weight', 'Purchase Dry Weight UOM', 'Purchase Wet Weight',
       'Purchase Wet Weight UOM', 'Receipt In Progress',
       'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

dff = dff.drop(columns = ['Sales Provisional Final2'])

dff.columns = ['Purchase Group Company Code', 'Purchase Trade ID',
       'Purchase Quota Titan ID', 'Purchase Assignment Reference',
       'Sales Assignment Reference', 'Purchase Traffic Operator',
       'Purchase Contractual Counterparty', 'Purchase Commodity',
       'Sales Assignment Counterparty', 'Sales Credit Security Type',
       'LC Reference', 'LC Original Amount', 'LC Current Balance',
       'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank',
       'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC',
       'LC Cover Available to SCR (SCR-Sales Contractual Rights)',
       'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID',
       'Sales Provisional Final',
       'Sales Provisional Number', 'Sales Provisional Percentage',
       'Sales Invoice Date', 'Sales Invoice Due Date',
       'Sales Invoice Days Overdue', 'Sales Invoice Counterparty',
       'Cost Code ID', 'Cost Amount(USD)', 'Invoice Line Quantity',
       'Invoice Line Quantity UOM', 'Invoice Line Amount(USD)', 'Paid Amount',
       'Sales Assignment Eligible Percentage', 'Authorised Paid Quantity', 'Financed Quantity', 'Financing Ratio',
       'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)',
       'Uninvoiced Final Receivables(USD)',
       'Summary Outstanding Receivables(USD)',
       'Sales Contractual Counterparty Open Account Approved',
       'Purchase Dry Weight', 'Purchase Dry Weight UOM', 'Purchase Wet Weight',
       'Purchase Wet Weight UOM', 'Receipt In Progress',
       'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight']

cols_ffca_sales = ['Sales Assignment ID ', 'Estimated Sales Valuation']
df_FFCA_s = pd.DataFrame(df_ffca, columns=cols_ffca_sales)
df_FFCA_s['Sales Assignment ID '] = df_FFCA_s['Sales Assignment ID '].astype(str)
df_final_s = pd.merge(dff, df_FFCA_s, left_on='Sales Assignment Reference', right_on='Sales Assignment ID ', how="left")
df_final_s = df_final_s.drop(columns=['Cost Amount(USD)', 'Sales Assignment ID '])
df_final_s.columns = ['Purchase Group Company Code', 'Purchase Trade ID',
       'Purchase Quota Titan ID', 'Purchase Assignment Reference',
       'Sales Assignment Reference', 'Purchase Traffic Operator',
       'Purchase Contractual Counterparty', 'Purchase Commodity',
       'Sales Assignment Counterparty', 'Sales Credit Security Type',
       'LC Reference', 'LC Original Amount', 'LC Current Balance',
       'LC Issuance Date', 'LC Expiry Date', 'LC Opening Bank',
       'LC Confirming Bank', 'Total Financeable Sales Invoices Covered by LC',
       'LC Cover Available to SCR (SCR-Sales Contractual Rights)',
       'SCR Covered By LC (SCR-Sales Contractual Rights)', 'Sales Invoice ID',
       'Sales Provisional Final', 'Sales Provisional Number',
       'Sales Provisional Percentage', 'Sales Invoice Date',
       'Sales Invoice Due Date', 'Sales Invoice Days Overdue',
       'Sales Invoice Counterparty', 'Cost Code ID',
       'Invoice Line Quantity', 'Invoice Line Quantity UOM',
       'Invoice Line Amount(USD)', 'Paid Amount',
       'Sales Assignment Eligible Percentage', 'Authorised Paid Quantity',
       'Financed Quantity', 'Financing Ratio',
       'Unpaid Provisional Receivables(USD)', 'Unpaid Final Receivables(USD)',
       'Uninvoiced Final Receivables(USD)',
       'Summary Outstanding Receivables(USD)',
       'Sales Contractual Counterparty Open Account Approved',
       'Purchase Dry Weight', 'Purchase Dry Weight UOM', 'Purchase Wet Weight',
       'Purchase Wet Weight UOM', 'Receipt In Progress',
       'Delivery In Progress', 'Eligible Dry Weight', 'Eligible Wet Weight',
       'Cost Amount(USD)']
df_final_s = df_final_s[order4]
df_final_s = df_final_s.drop_duplicates(['Purchase Assignment Reference'])
# df_final_s['Uninvoiced Final Receivables(USD)'] = df_final_s['Cost Amount(USD)'] - df_final_s['Invoice Line Amount(USD)']
# df_final_s['Unpaid Provisional Receivables(USD)'] = df_final_s['Unpaid Provisional Receivables(USD)_x']

def function_a(a):
   if a < 0:
      return 0
   else:
      return a

def function_Finqty(a,b):
   if a == 0 :
      return 0 
   else:
      return b
   
df_final_s['Uninvoiced Final Receivables(USD)']  = df_final_s.apply(lambda x: function_a(x['Uninvoiced Final Receivables(USD)']), axis = 1)
df_final_s['Financing ratio']  = df_final_s.apply(lambda x: function_Finqty(x['Financed Quantity'], x['Financing Ratio']), axis = 1)

# df_final_s['Summary Outstanding Receivables(USD)'] = df_final_s['Uninvoiced Final Receivables(USD)'] + df_final_s['Unpaid Final Receivables(USD)'] + df_final_s['Unpaid Provisional Receivables(USD)']

final_result = pd.concat([df_final_s, result4], ignore_index=True)

final_result = final_result.sort_values(by=['Sales Assignment Reference', 'Delivery In Progress'], ascending=[False, False], na_position= 'first') 
# final_result = final_result[order]
final_result = final_result[order4]
final_result['Purchase Group Company Code'] = 'PTE'
final_result[['Purchase Assignment Reference']] = final_result[['Purchase Assignment Reference']].astype('str')
final_result['Purchase Trade ID'] = final_result['Purchase Assignment Reference'].map(lambda x: str(x)[:6])
final_result.loc[:, 'Purchase Quota Titan ID'] = final_result['Purchase Assignment Reference'].apply(lambda st: st[st.rfind(".")-8: st.rfind(".")])

final_result = pd.merge(final_result, df4_x, on='Purchase Assignment Reference')


final_result['Financing Ratio'] = pd.to_numeric(final_result['Financing Ratio'], errors='coerce')
final_result = final_result.round({'Financing Ratio': 2})
final_result['Sales Provisional Final'] = np.nan

final_result['Sales Assignment Eligible Percentage'] = final_result['Sales Assignment Eligible Percentage'].apply(lambda x: format(x,".2%"))

c:\Users\Yuyan.Lu\Anaconda3\envs\Traf_TF\lib\site-packages\ipykernel_launcher.py:292: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\Yuyan.Lu\Anaconda3\envs\Traf_TF\lib\site-packages\ipykernel_launcher.py:293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\Users\Yuyan.Lu\Anaconda3\envs\Traf_TF\lib\site-packages\ipykernel_launcher.py:294: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

TypeError: function_a() takes 1 positional argument but 2 were given

In [ ]:
final_result.to_excel(r'K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0301\test_sss.xlsx')